In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import polars as pl
from sklearn.datasets import make_regression

from timelake import TimeLake
from timelake.constants import StorageType

In [2]:
# Params
n_hours = 24 * 30  # 30 days
n_features = 100

# Generate regression features
X, _ = make_regression(n_samples=n_hours, n_features=n_features)
timestamps = [datetime(2024, 1, 1) + timedelta(hours=i) for i in range(n_hours)]

# Create Polars DataFrame
df = pl.DataFrame(X, schema=[f"feature_{i}" for i in range(n_features)])
df = df.with_columns(pl.Series("date", timestamps))

In [3]:
PATH = Path("./my-test-timelake")

lake = TimeLake.create(
    path=PATH,
    df=df,
    timestamp_column="date",
)
df = lake.read()
df.shape

DeltaError: Generic error: A table already exists at: /Users/marcusgarsdal/Personal/timelake/notebooks/my-test-timelake/

In [ ]:
lake.read(start_date="2024-01-01", end_date="2024-01-03").shape

(72, 103)

In [ ]:
df_update = df[0:10]
lake.upsert(df_update)

In [ ]:
df = lake.read()
df.shape

(720, 103)

In [ ]:
df

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,date,date_day,inserted_at
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],str,datetime[μs]
-0.144268,-1.857755,0.543574,-1.435836,-0.675756,0.305048,-0.24695,-1.132194,-0.350304,1.276591,0.390961,0.412135,0.885243,1.87679,-1.970451,0.246818,0.190194,-0.474567,1.197974,-1.406458,0.051937,0.363343,-0.637009,-0.889926,1.108118,-1.048081,1.009039,0.955959,1.244892,-0.789147,-0.7128,-0.090019,-1.025324,-0.28729,-0.204436,-1.354189,0.711833,…,0.282654,-0.463919,-0.936559,-0.614608,-0.493558,-0.501905,-0.601898,-1.175316,-1.964375,0.791112,-0.184902,-0.845108,-0.475754,-0.341664,-1.173054,-1.515962,0.310603,-0.112173,0.070707,1.004718,-1.138748,1.265018,-1.169608,-0.70426,-0.180551,0.469773,-0.084751,0.343493,-0.718323,-1.359126,-0.400183,1.412842,-1.070131,1.319225,2024-01-11 16:00:00,"""2024-01-11""",2025-04-17 19:02:12.704233
-0.257991,0.43048,0.376801,-0.921056,0.489125,-0.842635,-0.133112,0.720871,-1.919723,0.514233,1.223128,-0.271341,0.604127,-0.622316,-1.067686,0.097602,-1.248424,-0.309185,0.745642,-0.223493,1.141362,-1.821556,-1.155267,2.178269,1.118815,-0.58252,1.003033,0.056147,1.088411,1.588092,-0.9305,0.99255,-0.643389,0.09347,0.09508,-0.687237,-1.11007,…,0.367846,0.99077,-1.105117,-0.374207,-1.475692,0.867758,1.207072,-2.01617,-0.31791,-1.112252,1.430654,-0.39034,-0.374689,0.114673,2.224875,-1.398329,-0.180082,0.149032,-1.04987,-1.657525,-2.050133,-0.178286,2.382676,0.633292,-0.590616,0.311924,0.596414,1.264399,0.206069,-0.135464,-0.847125,-0.210037,1.352323,-1.697973,2024-01-11 18:00:00,"""2024-01-11""",2025-04-17 19:02:12.704233
-0.439295,-0.700858,-0.312366,1.318895,1.684998,1.148801,0.749196,-0.125902,1.591382,1.151685,1.036098,-0.085302,0.633793,-0.596979,0.581901,-0.302647,1.936236,1.739209,0.321469,-0.613634,1.014886,1.612601,-0.661506,1.601596,1.610268,-0.137301,-1.718545,-0.594292,-0.505877,-1.039984,-0.849893,-0.238568,-0.302422,-0.524816,-0.465366,-1.45918,0.343428,…,1.132833,0.12219,0.308173,-0.623949,0.977351,-0.211635,0.081929,0.857664,0.785111,1.135293,0.524769,0.826458,1.73621,-0.938226,0.758083,0.271354,0.897993,1.415872,-1.827826,-0.125381,-0.041113,-0.426758,0.176838,1.125998,0.261031,-0.643637,0.578061,-0.304098,0.060636,0.584176,0.032286,0.03832,0.041469,-0.531159,2024-01-11 12:00:00,"""2024-01-11""",2025-04-17 19:02:12.704233
0.260502,0.967176,0.555212,-0.61504,0.76627,-0.925186,0.480897,-0.235919,0.812182,1.613933,-0.445129,0.549973,-0.609299,0.522792,0.278758,0.634589,0.543615,0.137902,-0.466583,1.209996,0.537671,-0.050027,-0.477772,-0.786519,-1.092286,-0.212257,-0.137671,-0.006566,0.381881,-0.506843,0.615316,-0.387729,0.110952,-0.545423,-0.360525,3.404713,0.735729,…,0.5701,0.818307,0.804803,-1.444816,1.322867,0.571716,-0.148453,-0.127829,-1.991979,-1.179303,-0.70273,-0.130235,0.240432,-0.129048,-1.790306,-0.626027,-1.437671,-1.207798,-0.902499,1.318494,-2.713516,-0.398022,1.509197,1.649137,-0.637024,0.354841,-0.957288,0.661199,0.475701,-1.4724,-0.641859,1.686563,-0.29677,-0.85631,2024-01-11 2

# We try to create a timelake on S3


In [4]:
import dotenv

dotenv.load_dotenv()
lake = TimeLake.create(
    path="s3://my-time-lake",
    df=df,
    timestamp_column="date",
    storage_type=StorageType.S3,
)

In [9]:
lake.read(start_date="2024-01-01", end_date="2024-01-03").shape

(72, 103)